# New FISP Presidential Project Scraper

In [1]:
# coding: utf-8

In [2]:
# import necessary python packages
import sys
#sys.path.append("/usr/local/lib/python2.7/site-packages")
import tweepy #https://github.com/tweepy/tweepy
import dropbox #https://www.dropbox.com/developers-v1/core/docs/python
import csv
import time
import os
from datetime import datetime
from collections import defaultdict
import logging
import gspread
import pandas as pd
import numpy as np
from openpyxl import load_workbook

#Twitter and Dropbox API credentials
import api_cred as ac

In [3]:
# setup debug logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Helper Functions

In [4]:
# depreceated function from when data was saved to a Google sheet
from oauth2client.service_account import ServiceAccountCredentials
def authenticate_gspread():
  # scopes that your application should be granted access
  scope = ['https://spreadsheets.google.com/feeds'] 
  # Create a Credentials object from the service account's credentials and the scopes
  credentials = ServiceAccountCredentials.from_json_keyfile_name('auth.json', scope)
  gc = gspread.authorize(credentials)
  return gc
  
# gets the list of cand or pac and returns it in a list
def gspread_get_lists(worksheet, is_cand):
  names = filter(lambda x: len(x) > 0, worksheet.col_values(2))
  max_ids = worksheet.col_values(3)[:len(names)]
  counts = worksheet.col_values(4)[:len(names)]
  indices = range(1,len(names)+1)
  lists = zip(names, max_ids, counts, indices)
  del lists[0] # the first one is column title
  return lists

In [5]:
from dropbox import DropboxOAuth2FlowNoRedirect
def authenticate_dropbox():
  auth_flow = DropboxOAuth2FlowNoRedirect(ac.APP_KEY, ac.APP_SECRET)
  
  authorize_url = auth_flow.start()
  print "1. Go to: " + authorize_url
  print "2. Click \"Allow\" (you might have to log in first)."
  print "3. Copy the authorization code."
  auth_code = raw_input("Enter the authorization code here: ").strip()
  
  try:
    oauth_result = auth_flow.finish(auth_code)
  except Exception, e:
    print ('Error: %s' % (e,))
    return
  
  dbx = dropbox.Dropbox(oauth_result.access_token)
  return dbx

In [6]:
def authenticate_twitter():
  auth = tweepy.OAuthHandler(ac.consumer_key, ac.consumer_secret)
  auth.set_access_token(ac.access_key, ac.access_secret)
  api = tweepy.API(auth)
  return api

In [7]:
def get_new_tweets(tweet_name, since_id):
  api = authenticate_twitter()
  tweets = []
  new_tweets = api.user_timeline(screen_name = tweet_name, since_id = since_id, count = 200)
  tweets.extend(new_tweets)
  if len(tweets) > 0:
    max_id = tweets[-1].id - 1
  while (len(new_tweets) > 0):
    new_tweets = api.user_timeline(screen_name = tweet_name, since_id = since_id, count = 200, max_id = max_id)
    tweets.extend(new_tweets)
    max_id = tweets[-1].id - 1
  
  tweets = [[tweet.id_str, tweet.created_at, tweet.text, "", "", "",tweet.retweet_count, tweet.favorite_count] for tweet in tweets]
  logger.info("Downloading %d tweets from %s" % (len(tweets), tweet_name))
  return tweets[::-1]

In [8]:
def get_lists(df):
  # put twitter handles, last acquired tweet ID, tweet count and store them in respective lists
  names = filter(lambda x: x > 0, df.iloc[:, 1])
  max_ids = df.iloc[:, 2]
  counts = df.iloc[:, 3]
  
  # save the number of entries
  indices = range(1,len(names)+1)
  
  lists = zip(names, max_ids, counts, indices)
  del lists[0] # the first one is column title
  return lists

In [9]:
# take the rows with multiple tweets checked and make an individual row for each tweets
def expand_lists(df):
  # create a list for each columns and a dict to later convert into an df
  id_ = []
  ratings = []
  sources = []
  tweets = {'id': id_, 'rating': ratings, 'source': sources}
  
  # loop thru each row and if tweet id is stored in a list then create df 
  # with each id in a separate row with its fact check data
  for index, row in df.iterrows():
    if (type(row[0]) == list):
      for i in row[0]:
        id_.append(i)
        ratings.append(row[1])
        sources.append(row[2])
      # drop the row containing multiple tweets
      df.drop(index, inplace=True)
  # create new df with tweets in their own row, then append them to the original dataframe
  new_df = pd.DataFrame(tweets)
  df = df.append(new_df)
      
  return df

In [10]:
def load_sheets(path):
  sheet_book = load_workbook(path)
  sheet_writer = pd.ExcelWriter(path, engine='openpyxl')
  sheet_writer.book = sheet_book
  sheet_writer.sheets = dict((ws.title, ws) for ws in sheet_book.worksheets)
  logger.info("Downloaded %s" % path)
  return sheet_writer

# Write to Sheets ↓

In [43]:
def collect_data(is_cand):
  start = time.time()
  logger.info("Start...")
  # dp_client = authenticate_dropbox()
  
  # set file pathway variables an expand to HOME
  path = '~/Dropbox/Summer_of_Tweets/working_sheets/'
  tweet_list = "Tweet_List.xlsx"
  cand_tweets = "Presidential_Tweets.xlsx"
  pac_tweets = "PAC_Tweets.xlsx"
  path = os.path.expanduser(path)
  
  # set sheet var to either pres or PAC
  if(is_cand):
    tweet_sheet = cand_tweets
    sheetname = 'candidate'
  else:
    tweet_sheet = pac_tweets
    sheetname = 'pac'
    
  # load and prepare list of twitter accounts    
  list_writer = load_sheets(path + tweet_list)
  list_df = pd.read_excel(path + tweet_list, sheetname=sheetname)
  list_df = list_df.dropna(thresh=4)
  # list_df['Last_Pulled'] = pd.to_datetime(list_df['Last_Pulled'], errors='coerce') 
  # properly load spreadsheet to append new data
  tweet_writer = load_sheets(path + tweet_sheet)
   
  # loop through the list of Cand/PACs and updates each tweet sheet appropriately
  for index, row in list_df.iterrows():       
    name, since_id, count = row[1], row[2],row[3]
    new_tweets = get_new_tweets(name, since_id)
    # if there are no new tweets continue to the next account
    if (len(new_tweets) > 0):
      # turn the new tweets into a dataframe and write them to the corresponding excel sheet
      df = pd.DataFrame(new_tweets)
      df.to_excel(tweet_writer, sheet_name=name, startrow=count+1, header=False, index=False)
  
      # update since_id, count, and last_pull date in tweet list
      list_df.iat[index,2] = new_tweets[len(new_tweets)-1][0] # since_id
      list_df.iat[index,3] = count + len(new_tweets) # last_pull
      list_df.iat[index,4] = pd.to_datetime(time.strftime("%m/%d/%Y %H:%M:%S"), errors='coerce') # last_pull date
      
      logger.info("Updated new tweets on spreadsheet for %s" % name)
      time.sleep(100)
  
  list_df.to_excel(list_writer, sheet_name=sheetname, index=False)
  tweet_writer.save()
  list_writer.save()
  
  logger.info("Done appending new tweets")
  end = time.time()
  logger.info("Time Elapsed: %d", float((end-start))/60)

In [44]:
collect_data(True)
# collect_data(False)

INFO:__main__:Start...
INFO:__main__:Downloaded /Users/SoloMune/Dropbox/Summer_of_Tweets/working_sheets/Tweet_List.xlsx
INFO:__main__:Downloaded /Users/SoloMune/Dropbox/Summer_of_Tweets/working_sheets/Presidential_Tweets.xlsx
INFO:__main__:Downloading 0 tweets from BernieSanders
INFO:__main__:Downloading 0 tweets from BobbyJindal
INFO:__main__:Downloading 0 tweets from CarlyFiorina
INFO:__main__:Downloading 0 tweets from ChrisChristie
INFO:__main__:Downloading 0 tweets from gov_gilmore
INFO:__main__:Downloading 1 tweets from GovernorPataki
INFO:__main__:Updated new tweets on spreadsheet for GovernorPataki
INFO:__main__:Downloading 0 tweets from GovernorPerry
INFO:__main__:Downloading 32 tweets from GovMikeHuckabee
INFO:__main__:Updated new tweets on spreadsheet for GovMikeHuckabee
INFO:__main__:Downloading 8 tweets from HillaryClinton
INFO:__main__:Updated new tweets on spreadsheet for HillaryClinton
INFO:__main__:Downloading 14 tweets from JebBush
INFO:__main__:Updated new tweets on s

In [33]:
# Params: is_cand - determines whether to pull candidates tweets or PAC tweets
# Purpose: Updates like and retweet totals
def collect_addition_data(is_cand):
  start = time.time()
  logger.info("Start...")
  # dp_client = authenticate_dropbox()
  
  # set file pathway variables an expand to HOME
  path = '~/Dropbox/Summer_of_Tweets/working_sheets/'
  tweet_list = "Tweet_List.xlsx"
  cand_tweets = "Presidential_Tweets.xlsx"
  pac_tweets = "PAC_Tweets.xlsx"
  path = os.path.expanduser(path)
  
  # set sheet var to either pres or PAC
  if(is_cand):
    tweet_sheet = cand_tweets
    sheetname = 'candidate'
  else:
    tweet_sheet = pac_tweets
    sheetname = 'pac'
  
  # load and prepare list of twitter accounts
  list_writer = load_sheets(path + tweet_list)
  list_df = pd.read_excel(path + tweet_list, sheetname=sheetname)
  list_df = list_df.dropna(thresh=4)
  # properly load spreadsheet to append new data
  tweet_writer = load_sheets(path + tweet_sheet)
  logger.info("Downloaded tweets list")
  
  # loop through the list of Cand/PACs and updates each tweet sheet appropriately
  for row in list_df.itertuples():       
    name, since_id, count = row[2], row[3],row[4]
    
    # read cand tweet sheet
    tweets_df = pd.read_excel(path + tweet_sheet, sheetname=name)
    logger.info("Retrived data from spreadsheet for %s" % name)
    
    # retreive updated tweets
    tweets = get_new_tweets(name, 1)
    updates_df = pd.DataFrame(tweets)
    
    # clean dataframe to only include id, retweets, and favorites
    updates_df = updates_df[[0, 6, 7]]
    updates_df.columns = ['id', 'retweets', 'favorites']
    
    tweets_df = update_metadata(tweets_df, updates_df)
    
    tweets_df.to_excel(tweet_writer, sheet_name=name, index=False)
    tweet_writer.save()
    logger.info("Updated data on spreadsheet for %s" % name)
    
    return 'done'
    
    time.sleep(100)
  logger.info("Done collecting additional data")

In [38]:
# figuring out how to update the retweet/like count using an apply function
def update_metadata(tweets_df, updates_df): 
  # convert tweet id to the same type as the updates sheet
  tweets_df['id'] = tweets_df['id'].astype(str)
  tweets_df.set_index('id', inplace=True)

  # loop through the updates metadata and updates the tweet sheet
  for row in updates_df.itertuples():
    tweets_df.set_value(row[1], 'retweets', row[2])
    tweets_df.set_value(row[1], 'favorites', row[3])

  # drop null rows that could not match with a tweet
  tweets_df.dropna(subset=['text'], inplace=True)
  tweets_df.reset_index(inplace=True)
  tweets_df['id'] = tweets_df['id'].astype(str)
  return tweets_df

In [23]:
collect_addition_data(True)
# collect_addition_data(False)

INFO:__main__:Start...
INFO:__main__:Downloaded /Users/SoloMune/Dropbox/Summer_of_Tweets/working_sheets/Tweet_List.xlsx
INFO:__main__:Downloaded /Users/SoloMune/Dropbox/Summer_of_Tweets/working_sheets/Presidential_Tweets.xlsx
INFO:__main__:Downloaded tweets list
INFO:__main__:Retrived data from spreadsheet for BernieSanders
INFO:__main__:Downloading 3215 tweets from BernieSanders
INFO:__main__:Updated data on spreadsheet for BernieSanders


'done'

In [ ]:
import requests
def get_full_url(short_urls, full_urls):
for i, us in enumerate(short_urls):
full = []
  if not us.startswith("http"):
    continue
  for url in us.split(" "):
    if not url.startswith("http"):
      continue
    try:
      r = requests.head(url, allow_redirects=True)
      full.append(r.url)
    except:
      logger.info("Error occurred for URL - %s" % url)
      continue
  if i % 500 == 0:
      logger.info("Extracting URL %d/%d" % (i, len(short_urls)))
      time.sleep(60)
  full_urls[i] = " ".join(full)q

In [24]:
def update_full_url(is_cand):
  start = time.time()
  logger.info("Start...")
  # dp_client = authenticate_dropbox()
  
  # set file pathway variables an expand to HOME
  path = '~/Dropbox/Summer_of_Tweets/working_sheets/'
  tweet_list = "Tweet_List.xlsx"
  cand_tweets = "Presidential_Tweets.xlsx"
  pac_tweets = "PAC_Tweets.xlsx"
  path = os.path.expanduser(path)
  
  # set sheet var to either pres or PAC
  if(is_cand):
    tweet_sheet = cand_tweets
    sheetname = 'candidate'
  else:
    tweet_sheet = pac_tweets
    sheetname = 'pac'
  
  # load and prepare list of twitter accounts
  list_writer = load_sheets(path + tweet_list)
  list_df = pd.read_excel(path + tweet_list, sheetname=sheetname)
  list_df = list_df.dropna(thresh=4)
  # properly load spreadsheet to append new data
  tweet_writer = load_sheets(path + tweet_sheet)
  logger.info("Downloaded tweets list")
    
  logger.info("Successfully download the list...")
  for e, entry in enumerate(lists):
    if e < 15:
      continue

    d = defaultdict(list)
    name, since_id, count, index = entry[0], entry[1],entry[2], entry[3]

    short_urls = worksheet.col_values(6)
    logger.info("Downloaded %s URL", name)
    url_datas = ['' for i in xrange(len(short_urls))]
    url_datas[0] = 'full URL'

    get_full_url(short_urls, url_datas) # transfer short url to full urls and store in url_datas

    count = 1

    while count < len(short_urls):
      amount = min(100, len(short_urls) - count)
      cells = worksheet.range('I'+str(count)+':'+'I'+str(count+amount-1))
      assert(len(cells) == amount)
      for i in range(amount):
        cells[i].value = url_datas[count-1]
        count += 1
      worksheet.update_cells(cells)
      logger.info("Update cells %d/%d for %s" %(count, len(short_urls), name))

In [ ]:
update_full_url(True)
#update_full_url(False)

The cell below collects fact checks from the Washington Post's '2016 Election Fact Checker' and 'RealDonaldContext' chrome extension. The election fact checker data was hand collected and is stored in a json file while the extension data is pulled directly from the online hosted json file from the extension's developer blog.
They are collected into an single dataframe consisting of the tweet id, rating, and source. They are then merged with a master sheet using tweet id.

['2016 Election Fact Checker'](https://www.washingtonpost.com/graphics/politics/2016-election/fact-checker/)

['RealDonaldContext'](https://chrome.google.com/webstore/detail/realdonaldcontext/ddbkmnomngnlcdglabflidgmhmcafogn?hl=en-US)

['RealDonaldContext json file'](https://www.pbump.net/files/post/extension/core/data.php)

In [19]:
# this code is from the fact checking portion of this project. It grabs the fact checked tweets from
# the WaPo Trump tweet fact checking extension and adds the ratings to correspoding tweets in the spreadsheet

# sheetnames
trump_sheet = 'realDonaldTrump'
potus_sheet = 'POTUS'

logger.info("Start...")
# modify print precison for easier debugging
np.set_printoptions(precision=20)

# read in WaPo fact checks of Donald Trump from the WaPo Trump tweet chrome extension
trump_check = pd.read_json('https://www.pbump.net/files/post/extension/core/data.php')
# rename columns and remove text columns
trump_check.columns = ['id', 'rating', 'tweet', 'source']
trump_check = trump_check[['id', 'rating', 'source']]
# call expand lists to turn fact checks of multiple tweets into multiple columns
trump_check = expand_lists(trump_check)

# load pre-election fact checks and filter for just id, rating, and source
election_checks = pd.read_json('preelection_wapo.json')
election_checks = election_checks[['id', 'rating', 'source']]

# append the hand collected data with the data collected from the extension
trump_check = trump_check.append(election_checks, ignore_index=True)
trump_check.columns = ['id', 'WAPO_RATING', 'WAPO_SOURCE']
logger.info("read in fact checks")

# set file pathway variables an expand to HOME
in_path = '~/Dropbox/Summer_of_Tweets/fact_checking/Presidential_Fact_Checking.xlsx'
in_path = os.path.expanduser(in_path)

# properly load spreadsheet to append new data
work_book = load_workbook(in_path)
tweet_writer = pd.ExcelWriter(in_path, engine='openpyxl')
tweet_writer.book = work_book
tweet_writer.sheets = dict((ws.title, ws) for ws in work_book.worksheets)
tweets_df = pd.read_excel(in_path, sheetname=trump_sheet, dtype={'id': str})
logger.info("Downloaded excel sheets list")

# change data type to match excel sheet's
trump_check['id'] = trump_check['id'].astype(str)
#merge the fact check data set with the tweets set using tweet id
merged_df = tweets_df.merge(trump_check, on='id', how='left')

logger.info(merged_df.shape) # used for debugging
# write merged data to the excel sheet
merged_df.to_excel(tweet_writer, sheet_name=trump_sheet, index=False)
tweet_writer.save()

# merged_df.to_csv('WaPo.csv', encoding='utf-8') # used for viewing test results

logger.info("done")

INFO:__main__:Start...
INFO:__main__:read in fact checks
INFO:__main__:Downloaded excel sheets list
INFO:__main__:(215, 15)
INFO:__main__:done


In [ ]:
# temp function to properly sort the tweets by date in ascending order
def sort_sheet (is_cand):
  start = time.time()
  logger.info("Start...")
  # dp_client = authenticate_dropbox()
      
  # set file pathway variables an expand to HOME
  path = '~/Dropbox/Summer_of_Tweets/working_sheets/'
  tweet_list = "Tweet_List.xlsx"
  cand_tweets = "Presidential_Tweets.xlsx"
  pac_tweets = "PAC_Tweets.xlsx"
  path = os.path.expanduser(path)
      
  # set sheet var to either pres or PAC
  if(is_cand):
    tweet_sheet = cand_tweets
    sheetname = 'candidate'
  else:
    tweet_sheet = pac_tweets
    sheetname = 'pac'
        
  # load and prepare list of twitter accounts    
  list_book = load_workbook(path + tweet_list)
  list_writer = pd.ExcelWriter(path + tweet_list, engine='openpyxl')
  list_writer.book = list_book
  list_writer.sheets = dict((ws.title, ws) for ws in list_book.worksheets)
  list_df = pd.read_excel(path + tweet_list, sheetname=sheetname)
  list_df = list_df.dropna(thresh=4)
  # properly load spreadsheet to append new data
  work_book = load_workbook(path + tweet_sheet)
  tweet_writer = pd.ExcelWriter(path + tweet_sheet, engine='openpyxl')
  tweet_writer.book = work_book
  tweet_writer.sheets = dict((ws.title, ws) for ws in work_book.worksheets)    
  logger.info("Downloaded tweets list")
       
  # loop through the list of Cand/PACs and updates each tweet sheet appropriately
  for index, row in list_df.iterrows():       
    name, since_id, count = row[1], row[2],row[3]
    tweets_df = pd.read_excel(path + tweet_sheet, sheetname=name)
    
    tweets_df = tweets_df.sort_values('id')